In [ ]:
# Current State - 
# 5 days 
# MAX_DATE = '2020-01-01'
# MIN_DATE = '2018-11-01'
# With normalization

# PM 0.6826923076923077


# ----------------------



In [1]:
import numpy as np
def get_possessions_per_game_and_team(row,pbp):
    try:
        GAME_ID = row['GAME_ID']
        is_home = row['IsHome']
        single_game = pbp[pbp['GAME_ID']==GAME_ID]
        index_indicator = 'Home_EOP' if is_home==1 else 'Visitor_EOP'

        single_game = single_game.sort_values('EVENTNUM')
        single_game['Shift_HOMEDESCRIPTION'] = single_game['HOMEDESCRIPTION'].shift(-1)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION']]
        single_game.loc[(single_game['HOMEDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_HOMEDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Home_EOP'] = 1
        single_game['Home_EOP'].fillna(0,inplace=True)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION','VISITORDESCRIPTION','Home_EOP']]

        single_game['Shift_VISITORDESCRIPTION'] = single_game['VISITORDESCRIPTION'].shift(-1)
        single_game[['Shift_VISITORDESCRIPTION','VISITORDESCRIPTION']]
        single_game.loc[(single_game['VISITORDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_VISITORDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Visitor_EOP'] = 1
        single_game['Visitor_EOP'].fillna(0,inplace=True)
        return single_game[['Home_EOP','Visitor_EOP']].sum().loc[index_indicator]
    except:
        return np.nan





In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import itertools



def load_dfs( MAX_DATE = None , MIN_DATE = None):
    nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
           'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
           'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
           'WAS', 'ORL', 'GSW', 'NOP', 
           'MIA', 
           'DET']
    nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
           1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
           1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
           1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
           1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
           1610612753, 1610612747, 1610612758, 1610612748, 1610612765]

   



    nba_teams_df = pd.DataFrame(nba_teams)
    dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')


    df = pd.read_csv('large_boxscoretraditionalv2_df.csv')
    df=df[df['TEAM_ID'].isin(nba_team_ids)]
    df.drop_duplicates(subset = ['GAME_ID','PLAYER_ID'],inplace=True)
    df.dropna(subset=['MIN'],inplace=True)



    df_g = pd.read_csv('all_games.csv')
    df_g.drop_duplicates(subset =['TEAM_ID','GAME_ID'],inplace=True)

    df_g.loc[df_g['WL']=='W','IsWin']=1
    df_g.loc[df_g['WL']=='L','IsWin']=0
    df_g['IsHome'] = (df_g['MATCHUP'].str.contains('vs.')).astype(int)
    df_g['Opposing_team'] = df_g['MATCHUP'].apply( lambda x: x.split(' ')[2])

    df_g=df_g[df_g['TEAM_ID'].isin(nba_team_ids)]
    team_abv_to_id_mapping = df_g[['TEAM_ABBREVIATION','TEAM_ID']].drop_duplicates().set_index('TEAM_ABBREVIATION')['TEAM_ID'].to_dict()
    df_g['Opposing_team_ID'] = df_g['Opposing_team'].apply(lambda x: team_abv_to_id_mapping[x] if x in team_abv_to_id_mapping else None )

    df_g=df_g[df_g['Opposing_team_ID'].isin(nba_team_ids)]

    # df_g=df_g[['GAME_ID','TEAM_ID','GAME_DATE']].drop_duplicates().dropna()
    df_g['GAME_DATE'] = pd.to_datetime(df_g['GAME_DATE'])
    
    if MAX_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']<MAX_DATE]
    if MIN_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']>MIN_DATE]
    # add balance scoring


    df = df.merge(df_g, 
         how='inner',
         left_on=['GAME_ID','TEAM_ID'],
         right_on=['GAME_ID','TEAM_ID'])


    pbp = pd.read_csv('large_playbyplayv2_df.csv', index_col=0)
    pbp.drop_duplicates(subset=['GAME_ID','EVENTNUM','PERIOD'], inplace=True)


    shot_chart_df =  pd.read_csv('ShotChartDetail_v2.csv')
    shot_chart_df.drop_duplicates(subset=['GAME_ID','GAME_EVENT_ID'], inplace=True)
    
    
    df_g['num_of_possessions'] = df_g.apply( lambda x: get_possessions_per_game_and_team(x,pbp), axis=1)
    df_g.dropna(subset=['num_of_possessions'],inplace=True)
    df_g['AdjustedPM'] = (df_g['PLUS_MINUS']/df_g['num_of_possessions'])*100
    df_g['OffRating'] = (df_g['PTS']/df_g['num_of_possessions'])*100
    df_g['EFG'] = (df_g['FGM'] + 0.5*df_g['FG3M'])/df_g['FGA']
    df_g['AST_ratio'] = df_g['AST']*100/((df_g['FGA'])+(df_g['FTA']*0.44)+(df_g['AST'])+(df_g['AST']))
    df_g['Opp_points'] = df_g['PTS'] + df_g['PLUS_MINUS']
    df_g['Def_Rating'] = (df_g['Opp_points'] / df_g['num_of_possessions'])*100

    ['AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']

    print('done')
    return df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping

df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs(  MAX_DATE = '2020-02-10',MIN_DATE = '2018-10-15')

done


In [3]:
nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
       'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
       'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
       'WAS', 'ORL', 'GSW', 'NOP', 
       'MIA', 
       'DET']
nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
       1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
       1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
       1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
       1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
       1610612753, 1610612747, 1610612758, 1610612748, 1610612765]





nba_teams_df = pd.DataFrame(nba_teams)
dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')

sliding_window_num_of_games = 5

large_df_of_sequences = pd.DataFrame([])
 
# create the sliding window dataframe per team
for team in nba_team_ids:
    print(team)
    x = df_g[df_g['TEAM_ID'] == team].sort_values('GAME_DATE').reset_index(drop=True)
    
    
    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values
    
    
    list_team_indicator = list(team_indicator) 
    for ix,row in x[sliding_window_num_of_games:].iterrows():
        tmp = x.loc[ix-sliding_window_num_of_games:ix]
        
        # add sequence of wins
        list_of_sequence_of_wins = list(tmp['IsWin'].values[:-1])
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
        df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        # get team indicator
        team_indicator = dummy_curr_team[0:1]
        
        # is at home
        list_of_sequence_of_at_home = list(tmp['IsHome'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games+1):
                new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
        df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
        
        # get opp team indicator 
        opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
        opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
        appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
        appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])
        
        
        # add averages of categories
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating'
                       ]
        averages_of_categories_curr_team = tmp[cols_to_calc][:-1].mean().values
        df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns =cols_to_calc)
        
        
        # add shot area avgs for curr team
        shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
       'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']==team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add shot chart of opposing teams during window
        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']!=team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add sequence of scores
        list_of_sequence_of_wins = list(tmp['PTS'].values[:-1])
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))
        
        df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        
         # add averages of categories for past opponents
        list_of_opp_ids = list(tmp['Opposing_team_ID'].unique())

        x_opp = df_g[(df_g['TEAM_ID'].isin(list_of_opp_ids))&
                    (df_g['GAME_DATE'] < tmp['GAME_DATE'].max())&
                    (df_g['GAME_DATE'] >= tmp['GAME_DATE'].min())].sort_values('GAME_DATE').reset_index(drop=True)
        tmp_opp = x_opp[-sliding_window_num_of_games:ix]
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
        averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
        df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns = cols_to_calc).reset_index(drop=True)
        
        # add sequence of pms
        list_of_sequence_of_pm = list(tmp['PLUS_MINUS'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_Plusminus'.format(game + 1))
        new_cols.append('Y') 
        df_of_sequence_of_plusminus = pd.DataFrame([list_of_sequence_of_pm],columns=new_cols)
        
        
        # add sequence of days between games
        tmp['GAME_DATE'].diff().dt.days.dropna()
        l = tmp['GAME_DATE'].diff().dt.days.dropna().values
        l = [4 if sl >4 else sl for sl in l]
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))

        df_days_between = pd.DataFrame([l],columns=new_cols)

        # start merging all together 
        # 1. wins
        # 2. team indicator
        # 3. at home indicator
        # 4. TOV sequence
#         print(len(team_indicator.columns),'team_indicator')
#         print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
#         print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
#         print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
#         print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
#         print(len(shot_area_avgs.columns),'shot_area_avgs')
#         print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
        
        df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_plusminus, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)

        large_df_of_sequences = large_df_of_sequences.append(df_all_features)
    
large_df_of_sequences.dropna(inplace=True)



1610612749
1610612766
1610612738
1610612746
1610612754
1610612750
1610612741
1610612742
1610612762
1610612759
1610612739
1610612752
1610612761
1610612760
1610612757
1610612751
1610612745
1610612756
1610612743
1610612755
1610612737
1610612763
1610612764
1610612744
1610612740
1610612753
1610612747
1610612758
1610612748
1610612765


In [8]:
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn import preprocessing



#scale the X variables
X = large_df_of_sequences.loc[:, large_df_of_sequences.columns != 'Y']
saved_columns = X.columns
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled, columns=saved_columns)




Y = large_df_of_sequences['Y']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

rf = RandomForestRegressor(n_estimators=1000, random_state=1)
rf.fit(x_train,y_train)


# Use the forest's predict method on the test data
predictions = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
    
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(X.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];    

y_predict = rf.predict(x_test)
# print ("R2 score:",r2_score(y_test,y_predict))


Mean Absolute Error: 10.26 degrees.
Accuracy: 97.16 %.
Variable: AdjustedPM_x         Importance: 0.0498
Variable: 1_games_back_IsHome  Importance: 0.0277
Variable: FT_PCT_x             Importance: 0.0197
Variable: Above the Break 3_y  Importance: 0.0186
Variable: FT_PCT_y             Importance: 0.0184
Variable: In The Paint (Non-RA)_y Importance: 0.0183
Variable: FG3_PCT_y            Importance: 0.0181
Variable: Above the Break 3_x  Importance: 0.0176
Variable: PLUS_MINUS_x         Importance: 0.0174
Variable: 4_games_back_Plusminus Importance: 0.0173
Variable: BLK_x                Importance: 0.0169
Variable: Mid-Range_x          Importance: 0.0166
Variable: Restricted Area_y    Importance: 0.0158
Variable: 1_games_back_PTS_x   Importance: 0.0152
Variable: Restricted Area_x    Importance: 0.0149
Variable: num_of_possessions_y Importance: 0.0149
Variable: FG3_PCT_x            Importance: 0.0148
Variable: 5_games_back_Plusminus Importance: 0.0143
Variable: In The Paint (Non-RA)_x Impo

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create param grid.

param_grid = {
    'bootstrap': [True],
    'max_depth': [150],
    'max_features': [200,100],
    'min_samples_leaf': [3],
    'min_samples_split': [ 3,6,12],
    'n_estimators': [500]
}
rf = RandomForestRegressor()

# Create grid search object

clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit on data

best_clf = clf.fit(x_train, y_train)
from sklearn.metrics import classification_report
print("Best parameters set found on development set:")
print(best_clf.best_params_)
print("Detailed classification report:")
y_true, y_pred = y_test, best_clf.predict(x_test)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  4.7min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  5.4min finished


Best parameters set found on development set:
{'bootstrap': True, 'max_depth': 150, 'max_features': 100, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 500}
Detailed classification report:


In [9]:
from sklearn.metrics import r2_score

def RFR(X_train, X_test, y_train, y_test, best_params):
    from sklearn.ensemble import RandomForestRegressor
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
    print ("R2 score:",r2_score(y_test,y_predict))
    return y_test,y_predict


# best_score, best_params = Grid_Search_CV_RFR(X_train, y_train)
y_test , y_predict = RFR(x_train, x_test, y_train, y_test ,best_clf.best_params_ )
print ("Best Score:" ,best_clf.best_score_)


R2 score: 0.13269282706603902
Best Score: 0.10099068623482245


In [10]:
import pickle
filename_pm = 'Plus_Minus_2018-11-01_2020-02-04_5Days.sav'
pickle.dump(best_clf, open(filename_pm,'wb'))
best_clf = pickle.load(open(filename_pm, 'rb'))



In [11]:

def predict_game_score(
                        curr_team_abv,
                        opp_team,
                        is_current_game_at_home,
                        curr_team_id,
                        df_g,
                        df,
                        pbp,
                        shot_chart_df
                        ):
    
    
    
    
    x = df_g[df_g['TEAM_ID'] == curr_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp = x[-sliding_window_num_of_games:]

    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values

    list_of_sequence_of_wins = list(tmp['IsWin'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
    df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)

    # # get team indicator
    team_indicator = dummy_curr_team[0:1]
    team_indicator

    # is at home
    list_of_sequence_of_at_home = list(tmp['IsHome'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
    df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
    df_of_sequence_of_at_home['{}_games_back_IsHome'.format(sliding_window_num_of_games+1)] = is_current_game_at_home


    # get opp team indicator 

    opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int).append(pd.Series(opp_team_id)), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
    appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
    appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])

    # add averages of categories
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_curr_team = tmp[cols_to_calc].mean().values
    df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns=cols_to_calc)
    df_of_averages_of_categories_curr_team

    # add shot area avgs for curr team
    shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
    'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']==curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs

    # add shot chart of opposing teams during window
    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']!=curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs_opp
    
    
    # add sequence of scores
    list_of_sequence_of_wins = list(tmp['PTS'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))
    df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)

        
     # add averages of categories
    x_opp = df_g[df_g['TEAM_ID'] == opp_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp_opp = x_opp[-sliding_window_num_of_games:]
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
    df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns=cols_to_calc)
    df_of_averages_of_categories_opp_team
    
    # add sequence of pms
    list_of_sequence_of_pm = list(tmp['PLUS_MINUS'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_Plusminus'.format(game + 1))
    df_of_sequence_of_plusminus = pd.DataFrame([list_of_sequence_of_pm],columns=new_cols)

   # add sequence of days between games
    tmp_days_back = x[-sliding_window_num_of_games-1:]
    l = tmp_days_back['GAME_DATE'].diff().dt.days.dropna().values
    l = [4 if sl >4 else sl for sl in l]
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))
    df_days_between = pd.DataFrame([l],columns=new_cols)


    # start merging all together 
    # 1. wins
    # 2. team indicator
    # 3. at home indicator
    # 4. TOV sequence
    # print(len(team_indicator.columns),'team_indicator')
    # print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
    # print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
    # print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
    # print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
    # print(len(shot_area_avgs.columns),'shot_area_avgs')
    # print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
    df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_plusminus, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)

    len(df_all_features.columns)
    df_all_features = df_all_features[X.columns]
#     df_all_features = df_all_features.loc[:, df_all_features.columns != 'Y']
    df_all_features = pd.DataFrame(min_max_scaler.transform(df_all_features),columns=df_all_features.columns)
    
#     len(X.columns)

    print([curr_team_abv,'------->' , best_clf.predict(df_all_features) , '-------',
                        opp_team,
                        is_current_game_at_home,
                        curr_team_id])
    print()
    results_saved = [single_date,curr_team_abv, best_clf.predict(df_all_features)[0] ,
                        opp_team,
                        is_current_game_at_home,
                        curr_team_id]
    return results_saved
    

In [16]:
# lsit_of_games = [
#                 ['TOR','POR'] , 
#                 ['CLE','DET'] , 
#                 ['BKN','OKC'] , 
#                 ['MEM','MIN'] , 
#                 ['PHX','SAC'] , 
#                 ['LAL','NYK'] , 
                
#             ]

# df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = '2020-01-07')

total_results_saved = []

FIRST_DATE_TO_CHECK = '2020-02-11'
LAST_DATE_TO_RUN_ON = '2020-02-13'



df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)

list_of_games_to_check = df_g_global[['GAME_DATE','TEAM_ABBREVIATION','GAME_ID','IsHome']]
len(list_of_games_to_check)
merged_list_of_games_to_check = list_of_games_to_check.merge(list_of_games_to_check,
                                                             how='inner',
                                                             left_on =['GAME_ID'] ,
                                                             right_on =['GAME_ID'])
merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['TEAM_ABBREVIATION_x']!= merged_list_of_games_to_check['TEAM_ABBREVIATION_y']]
merged_list_of_games_to_check=merged_list_of_games_to_check[merged_list_of_games_to_check['IsHome_x']==1]


merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']>FIRST_DATE_TO_CHECK]
list_of_dates = merged_list_of_games_to_check['GAME_DATE_x'].unique()
for single_date in list_of_dates:
    print (single_date)
    lsit_of_games = []
    for ix,row in merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']==single_date].iterrows():
        merged_list_of_games_to_check.iloc[1]
        game=[row['TEAM_ABBREVIATION_x'],row['TEAM_ABBREVIATION_y']]
        lsit_of_games.append(game)
    df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = str(single_date)[:10])
           
        
        
    
    sliding_window_num_of_games = 5

    for game in lsit_of_games:
        print(game)
    #     print(df_g['GAME_DATE'].max())
    #     df_g['TEAM_ABBREVIATION'].unique()
        print(game)
        print(df_g['GAME_DATE'].max())
        df_g['TEAM_ABBREVIATION'].unique()
        ###########################
        ### DEFINE YOUR TEAMS   ###
        ###########################
        curr_team_abv = game[0] #'PHX'
        curr_team_id = team_abv_to_id_mapping[curr_team_abv]

        opp_team = game[1] #'SAC'
        opp_team_id  = team_abv_to_id_mapping[opp_team]
        is_current_game_at_home = 1



        ###########################


        results_saved = predict_game_score(
                            curr_team_abv,
                           opp_team,
                           1,
                           curr_team_id ,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df
                            )
        total_results_saved.append(results_saved)
#     predict_game_score(
#                         opp_team,
#                        curr_team_abv,
#                        0,
#                        opp_team_id,
#                       df_g,
#                       df,
#                       pbp,
#                       shot_chart_df,
#                     X.columns)
    print()

done
2020-02-12T00:00:00.000000000
done
['DEN', 'LAL']
['DEN', 'LAL']
2020-02-11 00:00:00
['DEN', '------->', array([8.2196724]), '-------', 'LAL', 1, 1610612743]

['ORL', 'DET']
['ORL', 'DET']
2020-02-11 00:00:00
['ORL', '------->', array([-3.70271706]), '-------', 'DET', 1, 1610612753]

['CLE', 'ATL']
['CLE', 'ATL']
2020-02-11 00:00:00
['CLE', '------->', array([-6.15693976]), '-------', 'ATL', 1, 1610612739]

['UTA', 'MIA']
['UTA', 'MIA']
2020-02-11 00:00:00
['UTA', '------->', array([-6.91868551]), '-------', 'MIA', 1, 1610612762]

['NYK', 'WAS']
['NYK', 'WAS']
2020-02-11 00:00:00
['NYK', '------->', array([-3.44918713]), '-------', 'WAS', 1, 1610612752]

['MIN', 'CHA']
['MIN', 'CHA']
2020-02-11 00:00:00
['MIN', '------->', array([-3.7500371]), '-------', 'CHA', 1, 1610612750]

['BKN', 'TOR']
['BKN', 'TOR']
2020-02-11 00:00:00
['BKN', '------->', array([-0.49984818]), '-------', 'TOR', 1, 1610612751]

['MEM', 'POR']
['MEM', 'POR']
2020-02-11 00:00:00
['MEM', '------->', array([-0.9

In [17]:
pd.DataFrame(total_results_saved, columns=['Date','HomeTeam', 'OutcomePM', 'AwayTeam' , 'IsHome' , 'HomeTeamID' ]).to_csv('~/pm_Jan.csv',index=False, mode='a', header=False)

In [18]:
################
# RUN PREDICTED
################
# LAST_DATE_TO_RUN_ON = '2020-02-20'
single_date =  '2020-02-13'
LAST_DATE_TO_RUN_ON = '2020-02-13'

df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)
           
lsit_of_games = [
['BOS','LAC'],
['NOP','OKC'],
]


sliding_window_num_of_games = 5

for game in lsit_of_games:
    print(game)
#     print(df_g['GAME_DATE'].max())
#     df_g['TEAM_ABBREVIATION'].unique()
    print(game)
    print(df_g_global['GAME_DATE'].max())
    df_g['TEAM_ABBREVIATION'].unique()
    ###########################
    ### DEFINE YOUR TEAMS   ###
    ###########################
    curr_team_abv = game[0] #'PHX'
    curr_team_id = team_abv_to_id_mapping[curr_team_abv]

    opp_team = game[1] #'SAC'
    opp_team_id  = team_abv_to_id_mapping[opp_team]
    is_current_game_at_home = 1



    ###########################


    predict_game_score(
                        curr_team_abv,
                       opp_team,
                       1,
                       curr_team_id ,
                      df_g_global,
                      df,
                      pbp,
                      shot_chart_df)
   

done
['BOS', 'LAC']
['BOS', 'LAC']
2020-02-12 00:00:00
['BOS', '------->', array([0.37981542]), '-------', 'LAC', 1, 1610612738]

['NOP', 'OKC']
['NOP', 'OKC']
2020-02-12 00:00:00
['NOP', '------->', array([0.30641058]), '-------', 'OKC', 1, 1610612740]



In [25]:
pd.DataFrame(total_results_saved, columns=['Date','HomeTeam', 'OutcomePM', 'AwayTeam' , 'IsHome' , 'HomeTeamID' ])

,Date,HomeTeam,OutcomePM,AwayTeam,IsHome,HomeTeamID
0,2020-02-01,IND,6.432246,NYK,1,1610612754
1,2020-02-01,ORL,1.714237,MIA,1,1610612753
2,2020-02-01,LAC,2.919397,MIN,1,1610612746
3,2020-02-01,SAS,6.028633,CHA,1,1610612759
4,2020-02-01,WAS,-0.697588,BKN,1,1610612764
5,2020-02-01,POR,3.808655,UTA,1,1610612757
6,2020-02-01,DAL,0.055496,ATL,1,1610612742
7,2020-02-01,SAC,-2.411292,LAL,1,1610612758
8,2020-02-01,CLE,-7.263380,GSW,1,1610612739
9,2020-02-01,BOS,4.597735,PHI,1,1610612738
